Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [11]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import math

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [8]:
batch_size = 128
regularize_factor = 5e-4

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    regularizers = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
    loss += regularize_factor * regularizers
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [9]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 19.900110
Minibatch accuracy: 10.9%
Validation accuracy: 9.9%
Minibatch loss at step 500: 2.188264
Minibatch accuracy: 78.9%
Validation accuracy: 75.2%
Minibatch loss at step 1000: 1.729351
Minibatch accuracy: 77.3%
Validation accuracy: 77.0%
Minibatch loss at step 1500: 1.207322
Minibatch accuracy: 81.2%
Validation accuracy: 78.2%
Minibatch loss at step 2000: 1.003885
Minibatch accuracy: 84.4%
Validation accuracy: 78.8%
Minibatch loss at step 2500: 0.930565
Minibatch accuracy: 79.7%
Validation accuracy: 80.1%
Minibatch loss at step 3000: 0.889157
Minibatch accuracy: 79.7%
Validation accuracy: 80.8%
Test accuracy: 88.0%


In [12]:
IMAGE_PIXELS = image_size * image_size

batch_size = 128
hidden_units = 1024
regularize_factor = 5e-4

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size,
                                                         IMAGE_PIXELS))
    tf_train_labels = labels_placeholder = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    
    with tf.name_scope('hidden1'):
        h_weights = tf.Variable(
            tf.truncated_normal([image_size * image_size, hidden_units],
                            stddev=1.0 / math.sqrt(float(IMAGE_PIXELS))),
            name='weights')
        h_biases = tf.Variable(tf.zeros([hidden_units]),
                         name='biases')
        hidden = tf.nn.relu(tf.matmul(tf_train_dataset, h_weights) + h_biases)
        valid1 = tf.nn.relu(tf.matmul(tf_valid_dataset, h_weights) + h_biases)
        test1 = tf.nn.relu(tf.matmul(tf_test_dataset, h_weights) + h_biases)
        
    with tf.name_scope('softmax_linear'):
        weights = tf.Variable(
            tf.truncated_normal([hidden_units, num_labels],
                            stddev=1.0/ math.sqrt(float(IMAGE_PIXELS))),
            name = 'weights')
        biases = tf.Variable(tf.zeros([num_labels]))
        logits = tf.matmul(hidden, weights) + biases
        valid2 = tf.matmul(valid1, weights) + biases
        test2 = tf.matmul(test1, weights) + biases
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    regularizers = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases) +
                    tf.nn.l2_loss(h_weights) + tf.nn.l2_loss(h_biases))
    loss += regularize_factor * regularizers
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(valid2)
    test_prediction = tf.nn.softmax(test2)

In [13]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.505405
Minibatch accuracy: 7.8%
Validation accuracy: 41.7%
Minibatch loss at step 500: 0.561756
Minibatch accuracy: 87.5%
Validation accuracy: 85.6%
Minibatch loss at step 1000: 0.616280
Minibatch accuracy: 85.9%
Validation accuracy: 86.8%
Minibatch loss at step 1500: 0.445736
Minibatch accuracy: 92.2%
Validation accuracy: 87.4%
Minibatch loss at step 2000: 0.421554
Minibatch accuracy: 90.6%
Validation accuracy: 87.8%
Minibatch loss at step 2500: 0.433331
Minibatch accuracy: 89.1%
Validation accuracy: 87.9%
Minibatch loss at step 3000: 0.491320
Minibatch accuracy: 88.3%
Validation accuracy: 87.9%
Test accuracy: 94.0%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [14]:
batch_size = 128
regularize_factor = 5e-4

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    regularizers = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
    loss += regularize_factor * regularizers
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [18]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = ((step % 10) * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 17.750515
Minibatch accuracy: 10.2%
Validation accuracy: 11.0%
Minibatch loss at step 500: 1.570198
Minibatch accuracy: 91.4%
Validation accuracy: 69.6%
Minibatch loss at step 1000: 1.052739
Minibatch accuracy: 97.7%
Validation accuracy: 71.3%
Minibatch loss at step 1500: 0.797752
Minibatch accuracy: 99.2%
Validation accuracy: 72.1%
Minibatch loss at step 2000: 0.631073
Minibatch accuracy: 99.2%
Validation accuracy: 72.8%
Minibatch loss at step 2500: 0.511802
Minibatch accuracy: 99.2%
Validation accuracy: 73.4%
Minibatch loss at step 3000: 0.423318
Minibatch accuracy: 99.2%
Validation accuracy: 73.9%
Test accuracy: 81.1%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [19]:
IMAGE_PIXELS = image_size * image_size

batch_size = 128
hidden_units = 1024
regularize_factor = 5e-4

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size,
                                                         IMAGE_PIXELS))
    tf_train_labels = labels_placeholder = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    
    with tf.name_scope('hidden1'):
        h_weights = tf.Variable(
            tf.truncated_normal([image_size * image_size, hidden_units],
                            stddev=1.0 / math.sqrt(float(IMAGE_PIXELS))),
            name='weights')
        h_biases = tf.Variable(tf.zeros([hidden_units]),
                         name='biases')
        hidden = tf.nn.relu(tf.matmul(tf_train_dataset, h_weights) + h_biases)
        valid1 = tf.nn.relu(tf.matmul(tf_valid_dataset, h_weights) + h_biases)
        test1 = tf.nn.relu(tf.matmul(tf_test_dataset, h_weights) + h_biases)
        
    with tf.name_scope('softmax_linear'):
        weights = tf.Variable(
            tf.truncated_normal([hidden_units, num_labels],
                            stddev=1.0/ math.sqrt(float(IMAGE_PIXELS))),
            name = 'weights')
        biases = tf.Variable(tf.zeros([num_labels]))
        logits = tf.matmul(hidden, weights) + biases
        valid2 = tf.matmul(valid1, weights) + biases
        test2 = tf.matmul(test1, weights) + biases
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    regularizers = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases) +
                    tf.nn.l2_loss(h_weights) + tf.nn.l2_loss(h_biases))
    loss += regularize_factor * regularizers
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(valid2)
    test_prediction = tf.nn.softmax(test2)

In [20]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = ((step % 10) * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.507009
Minibatch accuracy: 10.9%
Validation accuracy: 39.2%
Minibatch loss at step 500: 0.197656
Minibatch accuracy: 100.0%
Validation accuracy: 80.7%
Minibatch loss at step 1000: 0.161843
Minibatch accuracy: 100.0%
Validation accuracy: 80.7%
Minibatch loss at step 1500: 0.135336
Minibatch accuracy: 100.0%
Validation accuracy: 80.7%
Minibatch loss at step 2000: 0.114825
Minibatch accuracy: 100.0%
Validation accuracy: 80.7%
Minibatch loss at step 2500: 0.098891
Minibatch accuracy: 100.0%
Validation accuracy: 80.8%
Minibatch loss at step 3000: 0.086470
Minibatch accuracy: 100.0%
Validation accuracy: 80.8%
Test accuracy: 87.7%


In [28]:
IMAGE_PIXELS = image_size * image_size

batch_size = 128
hidden_units = 1024
regularize_factor = 5e-4

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size,
                                                         IMAGE_PIXELS))
    tf_train_labels = labels_placeholder = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    
    with tf.name_scope('hidden1'):
        h_weights = tf.Variable(
            tf.truncated_normal([image_size * image_size, hidden_units],
                            stddev=1.0 / math.sqrt(float(IMAGE_PIXELS))),
            name='weights')
        h_biases = tf.Variable(tf.zeros([hidden_units]),
                         name='biases')
        hidden = tf.nn.relu(tf.matmul(tf_train_dataset, h_weights) + h_biases)
        valid1 = tf.nn.relu(tf.matmul(tf_valid_dataset, h_weights) + h_biases)
        test1 = tf.nn.relu(tf.matmul(tf_test_dataset, h_weights) + h_biases)
        hidden = tf.nn.dropout(hidden, 0.5)
        
    with tf.name_scope('softmax_linear'):
        weights = tf.Variable(
            tf.truncated_normal([hidden_units, num_labels],
                            stddev=1.0/ math.sqrt(float(IMAGE_PIXELS))),
            name = 'weights')
        biases = tf.Variable(tf.zeros([num_labels]))
        logits = tf.matmul(hidden, weights) + biases
        valid2 = tf.matmul(valid1, weights) + biases
        test2 = tf.matmul(test1, weights) + biases
    
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    regularizers = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases) +
                    tf.nn.l2_loss(h_weights) + tf.nn.l2_loss(h_biases))
    loss += regularize_factor * regularizers
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(valid2)
    test_prediction = tf.nn.softmax(test2)

In [29]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = ((step % 10) * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.618427
Minibatch accuracy: 8.6%
Validation accuracy: 39.9%
Minibatch loss at step 500: 0.213573
Minibatch accuracy: 100.0%
Validation accuracy: 81.0%
Minibatch loss at step 1000: 0.169710
Minibatch accuracy: 100.0%
Validation accuracy: 81.1%
Minibatch loss at step 1500: 0.141509
Minibatch accuracy: 100.0%
Validation accuracy: 80.8%
Minibatch loss at step 2000: 0.119928
Minibatch accuracy: 100.0%
Validation accuracy: 81.0%
Minibatch loss at step 2500: 0.102910
Minibatch accuracy: 100.0%
Validation accuracy: 81.1%
Minibatch loss at step 3000: 0.095489
Minibatch accuracy: 100.0%
Validation accuracy: 80.9%
Test accuracy: 88.0%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [55]:
IMAGE_PIXELS = image_size * image_size

batch_size = 128
hidden_units1 = 1024
hidden_units2 = 300
hidden_units3 = 50
regularize_factor = 5e-4

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size,
                                                         IMAGE_PIXELS))
    tf_train_labels = labels_placeholder = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    
    h1_weights = tf.Variable(
            tf.truncated_normal([image_size * image_size, hidden_units1],
                            stddev=1.0 / math.sqrt(float(IMAGE_PIXELS))),
            name='weights')
    h1_biases = tf.Variable(tf.zeros([hidden_units1]),
                         name='biases')
    hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, h1_weights) + h1_biases)
    valid1 = tf.nn.relu(tf.matmul(tf_valid_dataset, h1_weights) + h1_biases)
    test1 = tf.nn.relu(tf.matmul(tf_test_dataset, h1_weights) + h1_biases)
    hidden1 = tf.nn.dropout(hidden1, 0.5)
        
    h2_weights = tf.Variable(
            tf.truncated_normal([hidden_units1, hidden_units2],
                            stddev=1.0 / math.sqrt(float(IMAGE_PIXELS))),
            name='weights')
    h2_biases = tf.Variable(tf.zeros([hidden_units2]),
                         name='biases')
    hidden2 = tf.nn.relu(tf.matmul(hidden1, h2_weights) + h2_biases)
    valid2 = tf.nn.relu(tf.matmul(valid1, h2_weights) + h2_biases)
    test2 = tf.nn.relu(tf.matmul(test1, h2_weights) + h2_biases)
    hidden2 = tf.nn.dropout(hidden2, 0.5)
    
    h3_weights = tf.Variable(
            tf.truncated_normal([hidden_units2, hidden_units3],
                            stddev=1.0 / math.sqrt(float(IMAGE_PIXELS))),
            name='weights')
    h3_biases = tf.Variable(tf.zeros([hidden_units3]),
                         name='biases')
    hidden3 = tf.nn.relu(tf.matmul(hidden2, h3_weights) + h3_biases)
    valid3 = tf.nn.relu(tf.matmul(valid2, h3_weights) + h3_biases)
    test3 = tf.nn.relu(tf.matmul(test2, h3_weights) + h3_biases)
    hidden3 = tf.nn.dropout(hidden3, 0.5)
   
    weights = tf.Variable(
            tf.truncated_normal([hidden_units3, num_labels],
                            stddev=1.0/ math.sqrt(float(IMAGE_PIXELS))),
            name = 'weights')
    biases = tf.Variable(tf.zeros([num_labels]))
    logits = tf.matmul(hidden3, weights) + biases
    valid = tf.matmul(valid3, weights) + biases
    test = tf.matmul(test3, weights) + biases
    
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    regularizers = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases) +
                    tf.nn.l2_loss(h1_weights) + tf.nn.l2_loss(h1_biases) +
                    tf.nn.l2_loss(h2_weights) + tf.nn.l2_loss(h2_biases) + 
                    tf.nn.l2_loss(h3_weights) + tf.nn.l2_loss(h3_biases))
    loss += regularize_factor * regularizers
  
    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(0.5, global_step, train_labels.shape[0], 0.95)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(valid)
    test_prediction = tf.nn.softmax(test)

In [57]:
num_steps = 2001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.585361
Minibatch accuracy: 9.4%
Validation accuracy: 11.1%
Minibatch loss at step 500: 0.794842
Minibatch accuracy: 84.4%
Validation accuracy: 83.9%
Minibatch loss at step 1000: 0.742079
Minibatch accuracy: 85.2%
Validation accuracy: 84.8%
Minibatch loss at step 1500: 0.731989
Minibatch accuracy: 84.4%
Validation accuracy: 85.4%
Minibatch loss at step 2000: 0.618985
Minibatch accuracy: 89.1%
Validation accuracy: 85.8%
Test accuracy: 92.4%
